<a href="https://colab.research.google.com/github/vksriharsha/COMS673-Project/blob/main/Code/ICAlgorithm/IC_Algorithm_Accuracies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#### Imports ####

# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install causality
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install decorator>=4.1.2
!{sys.executable} -m pip install networkx>=2.0
!{sys.executable} -m pip install numpy>=1.13.3
!{sys.executable} -m pip install pandas>=0.20.3
!{sys.executable} -m pip install patsy>=0.4.1
!{sys.executable} -m pip install python-dateutil>=2.6.1
!{sys.executable} -m pip install pytz>=2017.2
!{sys.executable} -m pip install scipy>=0.19.1
!{sys.executable} -m pip install six>=1.11.0
!{sys.executable} -m pip install statsmodels>=0.8.0

!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pycm



^C


In [39]:

# causality imports
# IC Algorithm Tutorial: https://pypi.org/project/causality/
import numpy
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import time


from causality.inference.search import IC
from causality.inference.independence_tests import RobustRegressionTest

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score
from sklearn.metrics import confusion_matrix

from pycm import *


print('causality packages imported successfully')

causality packages imported successfully


In [40]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [41]:
import os

directory = '/content/drive/MyDrive/COM S 673/DREAM3 in silico challenge/Size50/DREAM3 data/'
df_index = []
time_df = pd.DataFrame(columns=['Time Taken'])

for filename in os.listdir(directory):
  if('Yeast' in filename or 'Ecoli' in filename):
    directory = directory+filename
    split_filename = (filename.split('-'))
    print(split_filename)
  
    if(split_filename[2] in ['null','nonoise']):
      type = split_filename[3].split('.')
      split_mix = split_filename[2]+'-'+type[0]
    else:
      type = split_filename[2].split('.')
      split_mix = type[0]

    df_index.append(split_filename[1]+'-'+split_mix)
    
df_columns = ['Accuracy']
print(df_index)
result_df = pd.DataFrame(columns=df_columns, index=df_index)
time_df = pd.DataFrame(columns=['Time Taken'], index = df_index)
result_df
time_df

['InSilicoSize50', 'Ecoli1', 'null', 'mutants.tsv']
['InSilicoSize50', 'Ecoli1', 'nonoise', 'heterozygous.tsv']
['InSilicoSize50', 'Ecoli1', 'nonoise', 'proteins', 'heterozygous.tsv']
['InSilicoSize50', 'Ecoli1', 'trajectories.tsv']
['InSilicoSize50', 'Ecoli1', 'heterozygous.tsv']
['InSilicoSize50', 'Ecoli2', 'nonoise', 'proteins', 'heterozygous.tsv']
['InSilicoSize50', 'Ecoli2', 'nonoise', 'heterozygous.tsv']
['InSilicoSize50', 'Ecoli2', 'heterozygous.tsv']
['InSilicoSize50', 'Ecoli2', 'null', 'mutants.tsv']
['InSilicoSize50', 'Ecoli2', 'trajectories.tsv']
['InSilicoSize50', 'Yeast1', 'null', 'mutants.tsv']
['InSilicoSize50', 'Yeast1', 'nonoise', 'heterozygous.tsv']
['InSilicoSize50', 'Yeast1', 'heterozygous.tsv']
['InSilicoSize50', 'Yeast1', 'nonoise', 'proteins', 'heterozygous.tsv']
['InSilicoSize50', 'Yeast1', 'trajectories.tsv']
['InSilicoSize50', 'Yeast2', 'nonoise', 'proteins', 'heterozygous.tsv']
['InSilicoSize50', 'Yeast2', 'null', 'mutants.tsv']
['InSilicoSize50', 'Yeast2', '

,Time Taken
Ecoli1-null-mutants,NaN
Ecoli1-nonoise-heterozygous,NaN
Ecoli1-nonoise-proteins,NaN
Ecoli1-trajectories,NaN
Ecoli1-heterozygous,NaN
Ecoli2-nonoise-proteins,NaN
Ecoli2-nonoise-heterozygous,NaN
Ecoli2-heterozygous,NaN
Ecoli2-null-mutants,NaN
Ecoli2-trajectories,NaN


In [42]:
import numpy as np

header_dct = {}

for i in range(1,51):
  header_dct['G'+str(i)] = 'c'

def print_theta_accuracy(standardnw_path,dataset_path):
  
  acc_value = []
  data = pd.read_csv(dataset_path, sep='\t') 
  data = data.iloc[: , 1:]


  benchmark_network = pd.read_csv(standardnw_path, sep='\t', header=None)
  benchmark_network = benchmark_network.loc[benchmark_network[2] == 1]


  benchmark_graph = nx.Graph()

  for i in (1,50):
      benchmark_graph.add_node('G'+str(i))

  for row in range(0,benchmark_network.shape[0]):
      benchmark_graph.add_edge(benchmark_network[0][row], benchmark_network[1][row])


  benchmark_adjMatrix = nx.adjacency_matrix(benchmark_graph)
  # Check time Elapsed for IC Algorithm to run
  # Time in seconds
  start = time.time()
  # run the IC search
  ic_algorithm = IC(RobustRegressionTest)
  predicted_graph = ic_algorithm.search(data, header_dct)
  end = time.time()
#   print(end - start)
  time_elapsed = end - start
#   time_list.append(time_elapsed)

  predicted_adjMatrix = nx.adjacency_matrix(predicted_graph)
  y_test = benchmark_adjMatrix.todense().flatten()
  y_pred = predicted_adjMatrix.todense().flatten()
  y_pred = np.asarray(y_pred)
  y_test = np.asarray(y_test)
  y_pred = y_pred.reshape(y_pred.shape[1],)
  y_test = y_test.reshape(y_test.shape[1],)

  print(y_pred.shape, y_test.shape)
  cm = ConfusionMatrix(y_test, y_pred)
  cmacc = [cm.ACC_Macro, cm.F1_Macro, cm.TPR_Macro, cm.FPR_Macro]
  cmacc = [round(num, 2) for num in cmacc]
#   acc_list.append(cm.ACC_Macro)
  
  return time_elapsed,cmacc;

In [43]:
directory = '/content/drive/MyDrive/COM S 673/DREAM3 in silico challenge/Size50/DREAM3 data/'
goldstandard = '/content/drive/MyDrive/COM S 673/DREAM3 in silico challenge/Size50/DREAM3 gold standards/'
df_columns = ['accuracy','timetaken']
df_index = os.listdir(directory)
filename_df = []
#result_df = pd.DataFrame(columns = df_columns, index = filename_df)
result_df = pd.DataFrame(columns = df_columns, index= df_index)


for filename in os.listdir(directory):
    data_path = directory+filename
    
    filename_temp  = filename
    split_filename = (filename_temp.split('-'))
    if(split_filename[2] == 'null'):
      type = split_filename[3].split('.')
    else:
      type = split_filename[2].split('.')
      split_mix = type[0]
        
    print(filename_temp, split_filename)

    index_location = split_filename[1]+'-'+split_mix

    filename_df = split_filename[0]+'-'+split_filename[1]+'-'+split_filename[2]
    print(filename_df)
    
    goldstandardname = ('DREAM3GoldStandard_'+split_filename[0]+'_'+split_filename[1]+'.txt')
    goldstandard_path = goldstandard+goldstandardname
    time_value, acc_value = print_theta_accuracy(goldstandard_path, data_path)
    print(filename+':')
    print(time_value, acc_value)
    print()
    result_df['accuracy'][filename] =  "ACC="+str(acc_value[0])+" , F1="+str(acc_value[1])+" , TPR="+str(acc_value[2])+" , FPR="+str(acc_value[3])
    result_df['timetaken'][filename] = time_value
    result_df.to_csv('/content/drive/MyDrive/COM S 673/DREAM3 in silico challenge/Results_IC/Size50/IC_results50_skeleton_only.csv')
    

InSilicoSize50-Ecoli1-null-mutants.tsv ['InSilicoSize50', 'Ecoli1', 'null', 'mutants.tsv']
InSilicoSize50-Ecoli1-null
(2500,) (2500,)
InSilicoSize50-Ecoli1-null-mutants.tsv:
416.3712909221649 [0.92, 0.51, 0.51, 0.49]

InSilicoSize50-Ecoli1-nonoise-heterozygous.tsv ['InSilicoSize50', 'Ecoli1', 'nonoise', 'heterozygous.tsv']
InSilicoSize50-Ecoli1-nonoise


/usr/local/lib/python3.7/dist-packages/statsmodels/robust/robust_linear_model.py:275: RuntimeWarning: divide by zero encountered in true_divide
  self.weights = self.M.weights(wls_results.resid/self.scale)
/usr/local/lib/python3.7/dist-packages/statsmodels/robust/robust_linear_model.py:275: RuntimeWarning: invalid value encountered in true_divide
  self.weights = self.M.weights(wls_results.resid/self.scale)


ValueError: ignored

In [ ]:
result_df